In [0]:
import keras
import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from itertools import count
from IPython.display import clear_output
from sklearn.metrics import accuracy_score
from keras.datasets import mnist
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten, Activation, Input, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.callbacks import EarlyStopping, CSVLogger
from scipy.stats import pearsonr
from tqdm import tqdm

In [0]:
BATCH_SIZE = 128
EPOCHS = 9999
IMAGE_SIZE = 28
NUM_CLASSES = 10
NR_OF_RUNS = 10
SEED_OFFSET = 10000
MODEL_NAME = "MNIST_baseline"
PATH = ""

# Preprocess

In [0]:
def preprocess(imgs):
    return imgs.reshape(imgs.shape[0], IMAGE_SIZE, IMAGE_SIZE, 1)

In [0]:
(x_train_val, y_train_val), (x_test, y_test) = mnist.load_data()

x_train_val = preprocess(x_train_val)
x_test = preprocess(x_test)

print('x_train shape:', x_train_val.shape)
print(x_train_val.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [0]:
# Convert class vectors to binary class matrices.
y_train_val = keras.utils.to_categorical(y_train_val, NUM_CLASSES)
y_testc = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [0]:
x_train_val = x_train_val.astype('float32')
x_test = x_test.astype('float32')
x_train_val /= 255
x_test /= 255

# Model

In [0]:
def MNISTmodel(imsize, num_classes, num_channels):
    inputs = Input((imsize,imsize,num_channels))
    x = Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', strides = 2)(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size = (2,2), strides=(2,2), padding = "same")(x)
    x = Conv2D(filters=32, kernel_size=(1,1), activation='relu', padding='valid')(x)
    x = Conv2D(filters = 10, kernel_size = (1,1),strides = (1,1), padding = 'valid')(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Activation('softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    optimizer = keras.optimizers.Adam(learning_rate = 1e-04)

    model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    return model

# Predict

In [0]:
def predict(model, X, Y):
    return accuracy_score(np.argmax(model.predict(X), axis=1), np.argmax(Y, axis=1))

# Train

In [0]:
for run in range(1, NR_OF_RUNS+1):

    # Split the data
    x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.20, shuffle= True)

    # Set the seeds
    np.random.seed(SEED_OFFSET + run)
    tf.random.set_seed(SEED_OFFSET + run)

    # Create directories
    os.makedirs(PATH + MODEL_NAME + f"/history", exist_ok=True)
    os.makedirs(PATH + MODEL_NAME + f"/weights", exist_ok=True)

    # Create the model
    model = MNISTmodel(IMAGE_SIZE, NUM_CLASSES, 1)
    
    # Load the weighs if the model is already trained
    weights_path = PATH + MODEL_NAME + f"/weights/weights-{run}.h5"

    es = EarlyStopping(min_delta=0.01, patience=3)
    csv_logger = CSVLogger(PATH + MODEL_NAME + f"/history/history-{run}.csv", separator=';')

    model.fit(x_train,y_train,
              batch_size = BATCH_SIZE,
              epochs = EPOCHS,
              validation_data = (x_val, y_val),
              shuffle = True,
              callbacks=[es, csv_logger])
    
    model.save_weights(weights_path)

    ## Accuracy
    ## The final accuracy on the test set
    print("\n ===== Accuracy ======")

    accuracy = predict(model, x_test, y_testc)
    print("Accuracy: " + str(accuracy))

    # Save the results
    file = PATH + MODEL_NAME + f"/results.csv"
    df = pd.DataFrame([[run, accuracy]])

    if not os.path.isfile(file):
      df.to_csv(file, header=["run", "accuracy"], index=False)
    else: # else it exists so append without writing the header
      df.to_csv(file, mode='a', header=False, index=False)

    clear_output(wait=True)